# DLO-JZ Optimisation de l'apprentissage - Jour 1 

Optimisation système d'une boucle d'apprentissage *Resnet-50*.

![car](./images/optimisation.png)

## Objet du notebook

Le but de ce *notebook* est d'optimiser un code d'apprentissage d'un modèle *Resnet-50* sur *Imagenet* pour Jean Zay en implémentant :
* **TP 1** : l'accélération GPU
* **TP 2** : l'*Automatic Mixed Precision*
* **TP 3** : le *Channels Last Memory Format*

Les cellules dans ce *notebook* ne sont pas prévues pour être modifiées, sauf rares exceptions indiquées dans les commentaires. Les TP se feront en modifiant le code `dlojz.py`.

Les directives de modification seront marquées par l'étiquette **TODO :**, dans le *notebook* suivant.
 
Les solutions sont présentes dans le répertoire `solutions`.

*Notebook rédigé par l'équipe assistance IA de l'IDRIS, juin 2023*


------------------------

### Environnement de calcul

Un module PyTorch doit avoir été chargé pour le bon fonctionnement de ce Notebook. **Nécessairement**, le module `pytorch-gpu/py3/1.11.0` :

In [ ]:
!module list

Les fonctions *python* de gestion de queue SLURM dévelopées par l'IDRIS et les fonctions dédiées à la formation DLO-JZ sont à importer.

Le module d'environnement pour les *jobs* et la taille des images sont fixés pour ce *notebook*.

**TODO :** choisir un *pseudonyme* (maximum 5 caractères) pour vous différencier dans la queue SLURM et dans les outils collaboratifs pendant la formation et la compétition.

In [ ]:
from idr_pytools import display_slurm_queue, gpu_jobs_submitter, search_log
from dlojz_tools import controle_technique, compare, GPU_underthehood, plot_accuracy, lrfind_plot, imagenet_starter
MODULE = 'pytorch-gpu/py3/1.11.0'
image_size = 176
account = 'for@v100'
name = 'pseudo'   ## Pseudonyme à choisir

Creation d'un repertoire `checkpoints` si cela n'a pas déjà été fait.

In [ ]:
!mkdir checkpoints

------------------------------------

### Gestion de la queue SLURM

Cette partie permet d'afficher et de gérer la queue SLURM.

Pour afficher toute la queue *utilisateur* :

In [ ]:
display_slurm_queue(name)

**Remarque**: Cette fonction utilisée plusieurs fois dans ce *notebook* permet d'afficher la queue de manière dynamique, rafraichie toutes les 5 secondes. Cependant elle ne s'arrête que lorsque la queue est vide. Si vous désirez reprendre la main sur le *notebook*, il vous suffira d'arrêter manuellement la cellule avec le bouton *stop*. Cela a bien sûr aucun impact sur le *scheduler* SLURM. Les *jobs* ne seront pas arrêtés.

Si vous voulez arrêter des *jobs* dans la queue:
* Annuler tous vos *jobs* dans la queue (décommenter la ligne suivante) : `!scancel -u $USER`
* Annuler un *job* dans votre queue (décommenter la ligne suivante et ajouter le numéro du *job* à la fin de la ligne)


In [ ]:
#!scancel -u $USER

------------------------------------

### Debug

Cette partie *debug* permet d'afficher les fichiers de sortie et les fichiers d'erreur du *job*.

Il est nécessaire dans la cellule suivante (en décommentant) d'indiquer le *jobid* correspondant sous le format suivant.

***Remarque*** : dans ce notebook, lorsque vous soumettrez un *job*, vous recevrez en retour le numéro du job dans le format suivant : `jobid = ['123456']`. La cellule ci-dessous peut ainsi être facilement actualisée."

In [ ]:
jobid = ['1493206']

Fichier de sortie :

In [ ]:
%cat {search_log(contains=jobid[0])[0]}

Fichier d'erreur :

In [ ]:
%cat {search_log(contains=jobid[0], with_err=True)['stderr'][0]}

--------------

### Différence entre deux scripts

Pour le *debug* ou pour comparer son code avec les solutions mises à disposition, la fonction suivante permet d'afficher une page html contenant un différentiel de fichiers texte.

In [ ]:
s1 = "dlojz.py"
s2 = "./solutions/dlojz1_3.py"
compare(s1, s2)

Voir le résultat du différentiel de fichiers sur la page suivante (attention au spoil !) :

[compare.html](compare.html)

------------------

## Dataset et modèle

Cette partie permet de visualiser les caractéristiques du *dataset* et du modèle utilisés.

### Imagenet

#### Train set

In [ ]:
import os
import torchvision
import torchvision.transforms as transforms
import torch
import numpy as np
import matplotlib.pyplot as plt

transform = transforms.Compose([ 
        transforms.RandomResizedCrop(176),             # Random resize - Data Augmentation
        transforms.RandomHorizontalFlip(),              # Horizontal Flip - Data Augmentation
        transforms.ToTensor(),                          # convert the PIL Image to a tensor
        transforms.Normalize(mean=(0.485, 0.456, 0.406),
                             std=(0.229, 0.224, 0.225))
        ])
    
    
train_dataset = torchvision.datasets.ImageNet(root=os.environ['ALL_CCFRSCRATCH']+'/imagenet',
                                                  transform=transform)
train_dataset

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,    
                                           batch_size=4,
                                           shuffle=True)
batch = next(iter(train_loader))
print('X train batch, shape: {}, data type: {}, Memory usage: {} bytes'
      .format(batch[0].shape, batch[0].dtype, batch[0].element_size()*batch[0].nelement()))
print('Y train batch, shape: {}, data type: {}, Memory usage: {} bytes'
      .format(batch[1].shape, batch[1].dtype, batch[1].element_size()*batch[1].nelement()))

img = batch[0][0].numpy().transpose((1,2,0))
plt.imshow(img)
plt.axis('off')
labels_cls, labels_id = torch.load(os.environ['ALL_CCFRSCRATCH']+'/imagenet/meta.bin')
label = labels_cls[np.unique(labels_id)[batch[1][0].numpy()]]
_ = plt.title('label class: {}'.format(label[0]))

#### Validation set

In [ ]:
val_transform = transforms.Compose([
                                    transforms.Resize((256, 256)),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),   # convert the PIL Image to a tensor
                                    transforms.Normalize(mean=(0.485, 0.456, 0.406),
                                    std=(0.229, 0.224, 0.225))])

val_dataset = torchvision.datasets.ImageNet(root=os.environ['ALL_CCFRSCRATCH']+'/imagenet', split='val',
                        transform=val_transform)
val_dataset

### Resnet-50

In [ ]:
import torchvision.models as models
model = models.resnet50()
print('number of total parameters: {}'.format(sum([p.numel() for p in model.parameters()])))
print('number of trainable parameters: {}'.format(sum([p.numel() for p in model.parameters() if p.requires_grad])))

--------------


# TP1_0 : Baseline CPU

Ce TP consiste à appliquer le code *baseline* pour prendre en main les fonctionnalités de test et découvrir le code.

**TODO :**
1. Exécuter les cellules suivantes (le *job* prend 7 minutes environ)
2. Puis, ouvrir le fichier `dlojz.py`

Remarque :
* l'option *test* lance un apprentissage de 50 itérations.
* les chronomètres mesurent les temps de la 10e à la 50e itération et restitue un temps moyen par itération.
* les parties `DON'T MODIFY` dans le script ne doivent pas être modifiées.

In [ ]:
n_gpu = 1
batch_size = 128

In [ ]:
command = f'dlojz.py -b {batch_size} --image-size {image_size} --test'
command

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobid = ['xxxxx']` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobid = ['1493206']

In [ ]:
display_slurm_queue(name)

In [ ]:
controle_technique(jobid)

Le code *baseline* `dlo-jz.py` a été exécuté sur le CPU (contrairement à ce qui est indiqué par le contrôle technique) en mode *test*, soit sur 50 itérations.

Dans le prochain exercice nous verrons ensemble l'accélération sur 1 GPU.


![Garage](images/stop.png "Arrêtez-vous ici! Une présentation vous attend avant le prochain TP.")

--------------------------------

# TP1_1 : Accélération GPU

**TODO** : dans le script `dlojz.py` :
* Définir la variable `gpu` et envoyer le modèle dans la mémoire du GPU.
```python
    # define model
    gpu = torch.device("cuda")
    model = models.resnet50()
    model = model.to(gpu)
```

* Envoyer les *batches* d'images d'entrée et les *labels* associés sur le GPU, pour **les étapes de *training* et de *validation***. *Remarque* : `non_blocking=args.non_blocking` sera utile plus tard lors de la définition d'un DataLoader distribué.

```python
    # distribution of images and labels to all GPUs
    images = images.to(gpu, non_blocking=args.non_blocking)
    labels = labels.to(gpu, non_blocking=args.non_blocking)
```
et

```python
    # distribution of images and labels to all GPUs
    val_images = val_images.to(gpu, non_blocking=args.non_blocking)
    val_labels = val_labels.to(gpu, non_blocking=args.non_blocking)
```

* Envoyer les tenseurs pour les métriques de validation sur le GPU.
```python
    ## Initialisation  
    if idr_torch.rank == 0: accuracies = []
    val_loss = torch.Tensor([0.]).to(gpu)                  # send to GPU
    val_accuracy = torch.Tensor([0.]).to(gpu)              # send to GPU
```


In [ ]:
n_gpu = 1
batch_size = 128
command = f'dlojz.py -b {batch_size} --image-size {image_size} --test'
command

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobid = ['xxxxx']` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobid = ['1493317']

In [ ]:
display_slurm_queue(name)

In [ ]:
controle_technique(jobid)

### Test d'occupation mémoire

Afin de mesurer l'impact de la taille de batch sur l'occupation mémoire et sur le *throughput*, la cellule suivante permet de soumettre plusieurs *jobs* avec des tailles de *batch* croissantes. Dans les cas où la mémoire est saturée et dépasse la capacité du GPU, le système renverra une erreur *CUDA Out of Memory*.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobids = ['xxxxx', ...]` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobids = ['1493424', '1493432', '1493433', '1493434', '1493435', '1493436', '1493438', '1493440']

In [ ]:
display_slurm_queue(name)

In [ ]:
GPU_underthehood(jobids)

Le dernier *job* a atteint le seuil *CUDA Out Of Memory* :

In [ ]:
controle_technique([jobids[-1]])

![Garage](images/stop.png "Arrêtez-vous ici! Une présentation vous attend avant le prochain TP.")

-------------------------------------

# TP1_2 : Automatic Mixed Precision

**TODO** : dans le script `dlojz.py` :
* Importer les fonctionnalités liées à l'*Automatic Mixed Precision*.
```python
from torch.cuda.amp import autocast, GradScaler
```

* Initialiser le *scaler*.
```python
    # Creates a GradScaler once at the beginning of training.
    scaler = GradScaler()
```    

* Implémenter l'*autocasting* (le changement de précision, FP32 à FP16) dans le *forward* , avec la ligne `with autocast():` dans la boucle de *training* **et** la boucle de validation.

```python
    ...
    # Runs the forward pass with autocasting.
    with autocast():
        outputs = model(images)
        loss = criterion(outputs, labels)
    ...
```

* Implémenter le *gradient scaling* pour la seule boucle de *training*. **Note**: À la place des lignes `loss.backward()` et `optimizer.step()`.

```python
    # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
    # Backward passes under autocast are not recommended.
    # Backward ops run in the same dtype autocast chose for corresponding forward ops.
    scaler.scale(loss).backward()

    # scaler.step() first unscales the gradients of the optimizer's assigned params.
    # If these gradients do not contain infs or NaNs, optimizer.step() is then called,
    # otherwise, optimizer.step() is skipped.
    scaler.step(optimizer)

    # Updates the scale for next iteration.
    scaler.update()

```


In [ ]:
n_gpu = 1
batch_size = 128
command = f'dlojz.py -b {batch_size} --image-size {image_size} --test'
command

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobid = ['xxxxx']` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobid = ['1493501']

In [ ]:
display_slurm_queue(name)

In [ ]:
controle_technique(jobid)

### Test d'occupation mémoire

Afin de mesurer l'impact de la taille de batch sur l'occupation mémoire et sur le *throughput*, la cellule suivante permet de soumettre plusieurs *jobs* avec des tailles de *batch* croissantes. Dans les cas où la mémoire est saturée et dépasse la capacité du GPU, le système renverra une erreur *CUDA Out of Memory*.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobids = ['xxxxx', ...]` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobids = ['1493508', '1493509', '1493511', '1493512', '1493513', '1493515', '1493516', '1493517']

In [ ]:
display_slurm_queue(name)

In [ ]:
GPU_underthehood(jobids)

### Changement de taille de batch

**TODO :** Choisir pour la suite du TP une taille de batch par GPU qui vous semble la plus pertinente selon le test précédent.

In [ ]:
## Choisir un batch size optimal
bs_optim = 512

In [ ]:
n_gpu = 1
command = f'dlojz.py -b {bs_optim} --image-size {image_size} --test'
command

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobids = ['xxxxx', ...]` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobid = ['1493563']

In [ ]:
display_slurm_queue(name)

In [ ]:
controle_technique(jobid)

![Commentaires](images/cedez.png "Assurez-vous que tout se passe bien avant de continuer!")

-----------------------------------------

# TP1_3 : Channels Last Memory Format

**TODO** : dans le script `dlojz.py` :
* Lors de l'envoie du modèle au GPU, configurer le paramètre `memory_format` avec `torch.channels_last`.
```python
    # define model
    gpu = torch.device("cuda")
    model = models.resnet50()
    model = model.to(gpu, memory_format=torch.channels_last)
```

* Lors de l'envoie des images d'entrée au GPU, configurer le paramètre `memory_format` avec `torch.channels_last`.

```python
    # distribution of images and labels to all GPUs
    images = images.to(gpu, non_blocking=args.non_blocking, memory_format=torch.channels_last)
    labels = labels.to(gpu, non_blocking=args.non_blocking)
```
et

```python
    # distribution of images and labels to all GPUs
    val_images = val_images.to(gpu, non_blocking=args.non_blocking, memory_format=torch.channels_last)
    val_labels = val_labels.to(gpu, non_blocking=args.non_blocking)
```

In [ ]:
n_gpu = 1
command = f'dlojz.py -b {bs_optim} --image-size {image_size} --test'
command

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobids = ['xxxxx', ...]` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobid = ['1493619']

In [ ]:
display_slurm_queue(name)

In [ ]:
controle_technique(jobid)

### Test d'occupation mémoire

Afin de mesurer l'impact de la taille de batch sur l'occupation mémoire et sur le *throughput*, la cellule suivante permet de soumettre plusieurs *jobs* avec des tailles de *batch* croissantes. Dans les cas où la mémoire est saturée et dépasse la capacité du GPU, le système renverra une erreur *CUDA Out of Memory*.

Soumission du *job*. **Attention vous sollicitez les noeuds de calcul à ce moment-là**.

Pour soumettre le job, veuillez basculer la cellule suivante du mode `Raw NBConvert` au mode `Code`.

Copier-coller la sortie `jobids = ['xxxxx', ...]` dans la cellule suivante.

Puis, rebasculer la cellule précédente en mode `Raw NBConvert`, afin d'eviter de relancer un job par erreur.

In [ ]:
#jobids = ['1493654', '1493656', '1493657', '1493658', '1493659', '1493660', '1493661', '1493662']

In [ ]:
display_slurm_queue(name)

In [ ]:
GPU_underthehood(jobids)

![Garage](images/stop.png "Arrêtez-vous ici! Une présentation vous attend avant le prochain TP.")

----------------------